In [16]:
!pip install selenium

In [17]:
from selenium import webdriver #permite criar o navegador
from selenium.webdriver.common.keys import Keys #permite escrever no navegador
from selenium.webdriver.common.by import By #permite selecionar itens no navegador

navegador = webdriver.Chrome()

# Passo 1: pegar a cotação do dólar
navegador.get('https://www.google.com.br/')

# - Pesquisar no google por "cotação dólar"
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# - Pegar a cotação no google
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)


# Passo 2: pegar a cotação do euro
navegador.get('https://www.google.com.br/')

# - Pesquisar no google por "cotação euro"
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# - Pegar a cotação no google
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)


# Passo 3: pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

navegador.quit()

5.2223999999999995
5.114688
288.92


In [18]:
# Passo 4: atualizar a base de preços (atualizando o preço de compra e o de venda)
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [19]:
# - Atualizar a coluna de cotação
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# - Atualizar a coluna de preço de compra = preço original * cotação
tabela['Preço de compra'] = tabela['Preço Original'] * tabela['Cotação']

# - Atualizar a coluna de preço de venda = preço de compra * margem
tabela['Preço de venda'] = tabela['Preço de compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda,Preço de compra,Preço de venda
0,Câmera Canon,999.99,Dólar,5.222400,4999.95,1.40,6999.930,5222.347776,7311.286886
1,Carro Renault,4500.00,Euro,5.114688,27000.00,2.00,54000.000,23016.096000,46032.192000
2,Notebook Dell,899.99,Dólar,5.222400,4499.95,1.70,7649.915,4700.107776,7990.183219
3,IPhone,799.00,Dólar,5.222400,3995.00,1.70,6791.500,4172.697600,7093.585920
4,Carro Fiat,3000.00,Euro,5.114688,18000.00,1.90,34200.000,15344.064000,29153.721600
5,Celular Xiaomi,480.48,Dólar,5.222400,2402.40,2.00,4804.800,2509.258752,5018.517504
6,Joia 20g,20.00,Ouro,288.920000,7000.00,1.15,8050.000,5778.400000,6645.160000


In [20]:
# Passo 5: exportar a base de preços atualizado
tabela.to_excel('Produtos Novo.xlsx', index=False)